## EV Profil Simulation

In [11]:
import numpy as np
import pandas as pd
import random

# chatgpt
## Diana

# Set constants
CHARGING_POWER_KW = 7.4  # Average home charger power (kW)
EFFICIENCY_KM_PER_KWH = 5  # Average distance (km) per kWh of battery
CHARGE_TIME_SLOT = [16, 7]  # Charging between 10 PM and 6 AM

## Define average EV profile for Germany
avg_distance_per_year_km = 13000
avg_distance_per_drive_km = 12
avg_battery_capacity_kwh = 72
avg_efficiency_Wh_per_100km = 191

## Homeoffice
driving_to_work_per_week = 3
driving_days_per_week = 5 
avg_distance_to_work_km = 22
avg_dayly_distance_km = 13

## kein Homeoffice
driving_to_work_per_week = 5
driving_days_per_week = 6 
avg_distance_to_work_km = 15
avg_dayly_distance_km = 22


# Define user profiles -> könnten genau abfragen & daraus ein neues Profil anlegen
users = [
    {"name": "kein Hommeoffice", "battery_capacity_kwh": avg_battery_capacity_kwh, "driving_days_per_week": 7, "avg_distance_per_day_km": 40},
    {"name": "Weekends", "battery_capacity_kwh": 50, "driving_days_per_week": 2, "avg_distance_per_day_km": 40},
    {"name": "Home-Office", "battery_capacity_kwh": 50, "driving_days_per_week": 3, "avg_distance_per_day_km": 40},
]

# Simulation period (e.g., 1 week)
days = 7
hours_per_day = 24

In [12]:

# Function to generate a user's driving pattern and charge availability
def generate_user_profile(user, days, hours_per_day):
    battery_capacity = user["battery_capacity_kwh"]
    driving_days = user["driving_days_per_week"]
    avg_distance = user["avg_distance_per_day_km"]
    
    # Driving pattern (randomly distribute driving days)
    driving_days_indices = sorted(random.sample(range(days), driving_days))
    
    battery_state = battery_capacity  # Start with a full battery
    consumption_per_day = avg_distance / EFFICIENCY_KM_PER_KWH  # kWh consumed per day of driving
    
    availability_matrix = np.ones((days, hours_per_day))  # Initialize all hours as available
    battery_profile = []  # Battery level per day
    
    for day in range(days):
        if day in driving_days_indices:
            # Simulate driving (reduce battery level)
            battery_state -= consumption_per_day
            if battery_state < 0:
                battery_state = 0  # Can't go below 0
                
            # Car is not available during driving hours (let's assume it's used from 8 AM to 6 PM)
            availability_matrix[day, 8:18] = 0  # Not available from 8 AM to 6 PM
        
        # Check for overnight charging (if the battery is not full)
        if battery_state < battery_capacity:
        # Charge overnight (10 PM to 6 AM)
            overnight_hours = list(range(CHARGE_TIME_SLOT[0], hours_per_day)) + list(range(0, CHARGE_TIME_SLOT[1]))
            for hour in overnight_hours:
                if battery_state < battery_capacity:
                    battery_state += CHARGING_POWER_KW / hours_per_day  # Increment battery level
                if battery_state > battery_capacity:
                    battery_state = battery_capacity
        
        battery_profile.append(battery_state)
    
    return availability_matrix, battery_profile


In [13]:

# Generate profiles for each user
profiles = {}
for user in users:
    availability_matrix, battery_profile = generate_user_profile(user, days, hours_per_day)
    profiles[user["name"]] = {"availability": availability_matrix, "battery": battery_profile}

# Convert to DataFrames for analysis
for user, data in profiles.items():
    availability_df = pd.DataFrame(data["availability"], columns=[f"Hour {i}" for i in range(hours_per_day)])
    battery_df = pd.DataFrame(data["battery"], columns=["Battery Level (kWh)"])
    print(f"{user}'s Charging Availability:\n", availability_df)
    print(f"{user}'s Battery Profile:\n", battery_df)


Comuter's Charging Availability:
    Hour 0  Hour 1  Hour 2  Hour 3  Hour 4  Hour 5  Hour 6  Hour 7  Hour 8  \
0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
1     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
2     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
3     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
4     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
5     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
6     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   

   Hour 9  ...  Hour 14  Hour 15  Hour 16  Hour 17  Hour 18  Hour 19  Hour 20  \
0     0.0  ...      0.0      0.0      0.0      0.0      1.0      1.0      1.0   
1     0.0  ...      0.0      0.0      0.0      0.0      1.0      1.0      1.0   
2     0.0  ...      0.0      0.0      0.0      0.0      1.0      1.0      1.0   
3     0.0  ...      0.0  

In [2]:
import numpy as np
import pandas as pd
import random

# Set constants for average EV profile in Germany
CHARGING_POWER_KW = 7.4  # Average home charger power (kW)
EFFICIENCY_WH_PER_KM = 191  # Efficiency in Wh per km (191 Wh/km = 19.1 kWh/100km)
EFFICIENCY_KWH_PER_KM = EFFICIENCY_WH_PER_KM / 1000  # Efficiency in kWh per km
AVG_BATTERY_CAPACITY_KWH = 72  # Average battery capacity in kWh

# Define time for charging (10 PM to 6 AM)
CHARGE_TIME_SLOT = [23, 0]

# Define the two profiles
profiles = {
    "Homeoffice": {
        "driving_to_work_per_week": 3,
        "driving_days_per_week": 5,
        "avg_distance_to_work_km": 22,
        "avg_daily_distance_km": 13,
    },
    "Kein Homeoffice": {
        "driving_to_work_per_week": 5,
        "driving_days_per_week": 6,
        "avg_distance_to_work_km": 15,
        "avg_daily_distance_km": 22,
    }
}

# Simulation period (1 week)
days = 7
hours_per_day = 24

# Function to simulate user driving and charging patterns
def generate_user_profile(profile_name, profile_data, days, hours_per_day):
    battery_capacity = AVG_BATTERY_CAPACITY_KWH
    driving_days = profile_data["driving_days_per_week"]
    driving_to_work_days = profile_data["driving_to_work_per_week"]
    avg_distance_to_work = profile_data["avg_distance_to_work_km"]
    avg_daily_distance = profile_data["avg_daily_distance_km"]
    
    # Driving pattern (randomly distribute driving days and workdays)
    driving_days_indices = sorted(random.sample(range(days), driving_days))
    work_days_indices = sorted(random.sample(driving_days_indices, driving_to_work_days))
    
    battery_state = battery_capacity  # Start with a full battery
    availability_matrix = np.ones((days, hours_per_day))  # Initialize all hours as available
    battery_profile = []  # Track battery level over time
    
    for day in range(days):
        # Determine how much distance is driven today
        if day in driving_days_indices:
            if day in work_days_indices:
                distance_driven = avg_distance_to_work * 2  # Round trip to work
            else:
                distance_driven = avg_daily_distance  # Normal day driving
            
            # Calculate battery consumption
            consumption_kwh = distance_driven * EFFICIENCY_KWH_PER_KM  # kWh used for the day
            battery_state -= consumption_kwh
            if battery_state < 0:
                battery_state = 0  # Can't go below 0
            
            # Car is not available during driving hours (8 AM to 6 PM)
            availability_matrix[day, 7:16] = 0
        
        # Charge overnight if battery is not full
        if battery_state < battery_capacity:
            overnight_hours = list(range(CHARGE_TIME_SLOT[0], hours_per_day)) + list(range(0, CHARGE_TIME_SLOT[1]))
            for hour in overnight_hours:
                if battery_state < battery_capacity*0.8:
                    battery_state += CHARGING_POWER_KW / hours_per_day  # Increment battery level
                    if battery_state > battery_capacity:
                        battery_state = battery_capacity
        
        battery_profile.append(battery_state)
    
    return availability_matrix, battery_profile

# Generate profiles for each user type
results = {}
for profile_name, profile_data in profiles.items():
    availability_matrix, battery_profile = generate_user_profile(profile_name, profile_data, days, hours_per_day)
    results[profile_name] = {"availability": availability_matrix, "battery": battery_profile}

# Convert results to DataFrames for better visualization
for profile_name, data in results.items():
    availability_df = pd.DataFrame(data["availability"], columns=[f"Hour {i}" for i in range(hours_per_day)])
    battery_df = pd.DataFrame(data["battery"], columns=["Battery Level (kWh)"])
    
    print(f"{profile_name}'s Charging Availability:\n", availability_df)
    print(f"{profile_name}'s Battery Profile:\n", battery_df)


Homeoffice's Charging Availability:
    Hour 0  Hour 1  Hour 2  Hour 3  Hour 4  Hour 5  Hour 6  Hour 7  Hour 8  \
0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0     0.0   
1     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0     0.0   
2     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0     0.0   
3     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0     0.0   
4     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
5     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
6     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0     0.0   

   Hour 9  ...  Hour 14  Hour 15  Hour 16  Hour 17  Hour 18  Hour 19  Hour 20  \
0     0.0  ...      0.0      0.0      1.0      1.0      1.0      1.0      1.0   
1     0.0  ...      0.0      0.0      1.0      1.0      1.0      1.0      1.0   
2     0.0  ...      0.0      0.0      1.0      1.0      1.0      1.0      1.0   
3     0.0  ...      0.

In [3]:
import numpy as np
import pandas as pd
import random

# Set constants for average EV profile in Germany
CHARGING_POWER_KW = 7.4  # Average home charger power (kW)
EFFICIENCY_WH_PER_KM = 191  # Efficiency in Wh per km (191 Wh/km = 19.1 kWh/100km)
EFFICIENCY_KWH_PER_KM = EFFICIENCY_WH_PER_KM / 1000  # Efficiency in kWh per km
AVG_BATTERY_CAPACITY_KWH = 72  # Average battery capacity in kWh

# Define battery limits (can't go below 10%, and charge up to 80%)
MIN_BATTERY_PERCENT = 0.10  # 10% minimum
MAX_BATTERY_PERCENT = 0.80  # 80% maximum

# Define the two profiles
profiles = {
    "Homeoffice": {
        "driving_to_work_per_week": 3,
        "driving_days_per_week": 5,
        "avg_distance_to_work_km": 22,
        "avg_daily_distance_km": 13,
    },
    "Kein Homeoffice": {
        "driving_to_work_per_week": 5,
        "driving_days_per_week": 6,
        "avg_distance_to_work_km": 15,
        "avg_daily_distance_km": 22,
    }
}

# Simulation period (1 week)
days = 7
hours_per_day = 24

# Function to simulate user driving and charging patterns
def generate_user_profile(profile_name, profile_data, days, hours_per_day):
    # Calculate min and max battery levels in kWh
    min_battery_level = AVG_BATTERY_CAPACITY_KWH * MIN_BATTERY_PERCENT
    max_battery_level = AVG_BATTERY_CAPACITY_KWH * MAX_BATTERY_PERCENT
    
    driving_days = profile_data["driving_days_per_week"]
    driving_to_work_days = profile_data["driving_to_work_per_week"]
    avg_distance_to_work = profile_data["avg_distance_to_work_km"]
    avg_daily_distance = profile_data["avg_daily_distance_km"]
    
    # Driving pattern (randomly distribute driving days and workdays)
    driving_days_indices = sorted(random.sample(range(days), driving_days))
    work_days_indices = sorted(random.sample(driving_days_indices, driving_to_work_days))
    
    battery_state = max_battery_level  # Start at the maximum battery level (80%)
    availability_matrix = np.ones((days, hours_per_day))  # Initialize all hours as available
    battery_profile = []  # Track battery level over time
    total_energy_used_for_charging = 0  # Track total energy used for charging
    
    for day in range(days):
        # Determine how much distance is driven today
        if day in driving_days_indices:
            if day in work_days_indices:
                distance_driven = avg_distance_to_work * 2  # Round trip to work
            else:
                distance_driven = avg_daily_distance  # Normal day driving
            
            # Calculate battery consumption
            consumption_kwh = distance_driven * EFFICIENCY_KWH_PER_KM  # kWh used for the day
            battery_state -= consumption_kwh
            
            # Ensure the battery doesn't go below 10% capacity
            if battery_state < min_battery_level:
                battery_state = min_battery_level  # Can't go below 10%
            
            # Car is not available during driving hours (8 AM to 6 PM)
            availability_matrix[day, 8:18] = 0  # Mark driving hours as unavailable
        
        # Charge at any time the car is available (all non-driving hours)
        for hour in range(hours_per_day):
            if availability_matrix[day, hour] == 1:  # Only charge if the car is available
                if battery_state < max_battery_level:  # Only charge up to 80%
                    energy_to_add = CHARGING_POWER_KW / hours_per_day  # Energy added in this hour
                    battery_state += energy_to_add
                    if battery_state > max_battery_level:
                        energy_to_add -= battery_state - max_battery_level  # Adjust energy added
                        battery_state = max_battery_level  # Cap battery level at 80%
                    total_energy_used_for_charging += energy_to_add  # Track energy added during charging
        
        battery_profile.append(battery_state)
    
    return availability_matrix, battery_profile, total_energy_used_for_charging

# Generate profiles for each user type
results = {}
for profile_name, profile_data in profiles.items():
    availability_matrix, battery_profile, total_energy_used_for_charging = generate_user_profile(
        profile_name, profile_data, days, hours_per_day
    )
    results[profile_name] = {
        "availability": availability_matrix, 
        "battery": battery_profile,
        "total_energy_used_for_charging": total_energy_used_for_charging
    }

# Convert results to DataFrames for better visualization
for profile_name, data in results.items():
    availability_df = pd.DataFrame(data["availability"], columns=[f"Hour {i}" for i in range(hours_per_day)])
    battery_df = pd.DataFrame(data["battery"], columns=["Battery Level (kWh)"])
    
    print(f"{profile_name}'s Charging Availability:\n", availability_df)
    print(f"{profile_name}'s Battery Profile:\n", battery_df)
    print(f"Total energy used for charging {profile_name}: {data['total_energy_used_for_charging']:.2f} kWh\n")


Homeoffice's Charging Availability:
    Hour 0  Hour 1  Hour 2  Hour 3  Hour 4  Hour 5  Hour 6  Hour 7  Hour 8  \
0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
1     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
2     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
3     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
4     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
5     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
6     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   

   Hour 9  ...  Hour 14  Hour 15  Hour 16  Hour 17  Hour 18  Hour 19  Hour 20  \
0     0.0  ...      0.0      0.0      0.0      0.0      1.0      1.0      1.0   
1     1.0  ...      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
2     1.0  ...      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
3     0.0  ...      0.

In [4]:
import numpy as np
import pandas as pd
import random

# Set constants for average EV profile in Germany
CHARGING_POWER_KW = 7.4  # Average home charger power (kW)
EFFICIENCY_WH_PER_KM = 191  # Efficiency in Wh per km (191 Wh/km = 19.1 kWh/100km)
EFFICIENCY_KWH_PER_KM = EFFICIENCY_WH_PER_KM / 1000  # Efficiency in kWh per km
AVG_BATTERY_CAPACITY_KWH = 72  # Average battery capacity in kWh

# Define battery limits (can't go below 10%, and charge up to 80%)
MIN_BATTERY_PERCENT = 0.10  # 10% minimum
MAX_BATTERY_PERCENT = 0.80  # 80% maximum

# Define the two profiles
profiles = {
    "Homeoffice": {
        "driving_to_work_per_week": 3,
        "driving_days_per_week": 5,
        "avg_distance_to_work_km": 22,
        "avg_daily_distance_km": 13,
    },
    "Kein Homeoffice": {
        "driving_to_work_per_week": 5,
        "driving_days_per_week": 6,
        "avg_distance_to_work_km": 15,
        "avg_daily_distance_km": 22,
    }
}

# Simulation period (1 week)
days = 7
hours_per_day = 24

# Function to simulate user driving and charging patterns
def generate_user_profile(profile_name, profile_data, days, hours_per_day):
    # Calculate min and max battery levels in kWh
    min_battery_level = AVG_BATTERY_CAPACITY_KWH * MIN_BATTERY_PERCENT
    max_battery_level = AVG_BATTERY_CAPACITY_KWH * MAX_BATTERY_PERCENT
    
    driving_days = profile_data["driving_days_per_week"]
    driving_to_work_days = profile_data["driving_to_work_per_week"]
    avg_distance_to_work = profile_data["avg_distance_to_work_km"]
    avg_daily_distance = profile_data["avg_daily_distance_km"]
    
    # Driving pattern (randomly distribute driving days and workdays)
    driving_days_indices = sorted(random.sample(range(days), driving_days))
    work_days_indices = sorted(random.sample(driving_days_indices, driving_to_work_days))
    
    battery_state = max_battery_level  # Start at the maximum battery level (80%)
    availability_matrix = np.ones((days, hours_per_day))  # Initialize all hours as available
    battery_profile = []  # Track battery level over time
    total_energy_used_for_charging = 0  # Track total energy used for charging
    total_km_driven = 0  # Track total kilometers driven
    
    for day in range(days):
        # Determine how much distance is driven today
        if day in driving_days_indices:
            if day in work_days_indices:
                distance_driven = avg_distance_to_work * 2  # Round trip to work
            else:
                distance_driven = avg_daily_distance  # Normal day driving
            
            total_km_driven += distance_driven  # Add to total kilometers driven
            
            # Calculate battery consumption
            consumption_kwh = distance_driven * EFFICIENCY_KWH_PER_KM  # kWh used for the day
            battery_state -= consumption_kwh
            
            # Ensure the battery doesn't go below 10% capacity
            if battery_state < min_battery_level:
                battery_state = min_battery_level  # Can't go below 10%
            
            # Car is not available during driving hours (8 AM to 6 PM)
            availability_matrix[day, 8:18] = 0  # Mark driving hours as unavailable
        
        # Charge at any time the car is available (all non-driving hours)
        for hour in range(hours_per_day):
            if availability_matrix[day, hour] == 1:  # Only charge if the car is available
                if battery_state < max_battery_level:  # Only charge up to 80%
                    energy_to_add = CHARGING_POWER_KW / hours_per_day  # Energy added in this hour
                    battery_state += energy_to_add
                    if battery_state > max_battery_level:
                        energy_to_add -= battery_state - max_battery_level  # Adjust energy added
                        battery_state = max_battery_level  # Cap battery level at 80%
                    total_energy_used_for_charging += energy_to_add  # Track energy added during charging
        
        battery_profile.append(battery_state)
    
    return availability_matrix, battery_profile, total_energy_used_for_charging, total_km_driven

# Generate profiles for each user type
results = {}
for profile_name, profile_data in profiles.items():
    availability_matrix, battery_profile, total_energy_used_for_charging, total_km_driven = generate_user_profile(
        profile_name, profile_data, days, hours_per_day
    )
    results[profile_name] = {
        "availability": availability_matrix, 
        "battery": battery_profile,
        "total_energy_used_for_charging": total_energy_used_for_charging,
        "total_km_driven": total_km_driven
    }

# Convert results to DataFrames for better visualization
for profile_name, data in results.items():
    availability_df = pd.DataFrame(data["availability"], columns=[f"Hour {i}" for i in range(hours_per_day)])
    battery_df = pd.DataFrame(data["battery"], columns=["Battery Level (kWh)"])
    
    print(f"{profile_name}'s Charging Availability:\n", availability_df)
    print(f"{profile_name}'s Battery Profile:\n", battery_df)
    print(f"Total energy used for charging {profile_name}: {data['total_energy_used_for_charging']:.2f} kWh")
    print(f"Total kilometers driven by {profile_name}: {data['total_km_driven']:.2f} km\n")


Homeoffice's Charging Availability:
    Hour 0  Hour 1  Hour 2  Hour 3  Hour 4  Hour 5  Hour 6  Hour 7  Hour 8  \
0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
1     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
2     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
3     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
4     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
5     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
6     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   

   Hour 9  ...  Hour 14  Hour 15  Hour 16  Hour 17  Hour 18  Hour 19  Hour 20  \
0     0.0  ...      0.0      0.0      0.0      0.0      1.0      1.0      1.0   
1     1.0  ...      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
2     0.0  ...      0.0      0.0      0.0      0.0      1.0      1.0      1.0   
3     1.0  ...      1.

In [1]:
import numpy as np
import pandas as pd
import random

# Set constants for average EV profile in Germany
CHARGING_POWER_KW = 7.4  # Average home charger power (kW)
EFFICIENCY_WH_PER_KM = 191  # Efficiency in Wh per km (191 Wh/km = 19.1 kWh/100km)
EFFICIENCY_KWH_PER_KM = EFFICIENCY_WH_PER_KM / 1000  # Efficiency in kWh per km
AVG_BATTERY_CAPACITY_KWH = 72  # Average battery capacity in kWh

# Define battery limits (can't go below 10%, and charge up to 80%)
MIN_BATTERY_PERCENT = 0.10  # 10% minimum
MAX_BATTERY_PERCENT = 0.80  # 80% maximum

# Define the two profiles
profiles = {
    "Homeoffice": {
        "driving_to_work_per_week": 3,  # days with 22 km commute (round trip)
        "driving_days_per_week": 5,     # weekdays
        "avg_one_way_distance_km": 12,  # 12 km on non-workdays (one-way)
    },
    "Kein Homeoffice": {
        "driving_days_per_week": 5,     # weekdays
        "avg_daily_distance_km": 22,    # 22 km total (no distinction between work/non-work)
    }
}

# Simulation period (1 week)
days = 7
hours_per_day = 24

# Weekend configuration (12 km for short trips, randomize longer vacation trips)
WEEKEND_MIN_DISTANCE = 12
WEEKEND_DAYS = [5, 6]  # Saturday and Sunday are day indices 5 and 6

# Function to simulate user driving and charging patterns
def generate_user_profile(profile_name, profile_data, days, hours_per_day):
    # Calculate min and max battery levels in kWh
    min_battery_level = AVG_BATTERY_CAPACITY_KWH * MIN_BATTERY_PERCENT
    max_battery_level = AVG_BATTERY_CAPACITY_KWH * MAX_BATTERY_PERCENT
    
    driving_days = profile_data.get("driving_days_per_week", 5)
    driving_to_work_days = profile_data.get("driving_to_work_per_week", 0)
    avg_one_way_distance_km = profile_data.get("avg_one_way_distance_km", 0)
    avg_daily_distance_km = profile_data.get("avg_daily_distance_km", 0)
    
    # Weekday driving pattern (randomly distribute driving days)
    driving_days_indices = sorted(random.sample(range(5), driving_days))  # Only weekdays (Mon-Fri)
    work_days_indices = sorted(random.sample(driving_days_indices, driving_to_work_days)) if "Homeoffice" in profile_name else []
    
    # Randomly assign driving on weekends (Saturday and/or Sunday)
    weekend_driving_days = [day for day in WEEKEND_DAYS if random.choice([True, False])]
    
    battery_state = max_battery_level  # Start at the maximum battery level (80%)
    availability_matrix = np.ones((days, hours_per_day))  # Initialize all hours as available
    battery_profile = []  # Track battery level over time
    total_energy_used_for_charging = 0  # Track total energy used for charging
    total_km_driven = 0  # Track total kilometers driven
    
    for day in range(days):
        # Weekday (Monday to Friday)
        if day < 5:
            if profile_name == "Homeoffice":
                if day in work_days_indices:
                    distance_driven = avg_one_way_distance_km * 2  # 22 km round trip on workdays
                else:
                    distance_driven = avg_one_way_distance_km * 2  # Non-workday: 24 km total
            else:  # Kein Homeoffice (22 km fixed every weekday)
                distance_driven = avg_daily_distance_km

            total_km_driven += distance_driven  # Add to total kilometers driven
            
            # Calculate battery consumption
            consumption_kwh = distance_driven * EFFICIENCY_KWH_PER_KM  # kWh used for the day
            battery_state -= consumption_kwh
            
            # Ensure the battery doesn't go below 10% capacity
            if battery_state < min_battery_level:
                battery_state = min_battery_level  # Can't go below 10%
            
            # Car is not available during driving hours (8 AM to 6 PM)
            availability_matrix[day, 8:18] = 0  # Mark driving hours as unavailable
        
        # Weekend (Saturday or Sunday)
        elif day in weekend_driving_days:
            # Decide if it's a short trip (12 km) or a long trip (up to battery limit)
            if random.choice([True, False]):
                # Short trip (12 km)
                distance_driven = WEEKEND_MIN_DISTANCE
            else:
                # Longer trip (vacation) until battery reaches 10%
                distance_driven = 0
                while battery_state > min_battery_level:
                    distance_driven += 1
                    battery_state -= EFFICIENCY_KWH_PER_KM
                    if battery_state < min_battery_level:
                        battery_state = min_battery_level
                        break
            
            total_km_driven += distance_driven  # Add weekend kilometers driven
            
            # Calculate battery consumption for the weekend drive
            consumption_kwh = distance_driven * EFFICIENCY_KWH_PER_KM
            battery_state -= consumption_kwh
            
            # Ensure battery doesn't go below 10%
            if battery_state < min_battery_level:
                battery_state = min_battery_level
            
            # Simulate the car being unavailable during driving hours on weekends
            availability_matrix[day, 10:14] = 0  # Random driving time, e.g., 10 AM to 2 PM
        
        # Charge at any time the car is available (all non-driving hours)
        for hour in range(hours_per_day):
            if availability_matrix[day, hour] == 1:  # Only charge if the car is available
                if battery_state < max_battery_level:  # Only charge up to 80%
                    energy_to_add = CHARGING_POWER_KW / hours_per_day  # Energy added in this hour
                    battery_state += energy_to_add
                    if battery_state > max_battery_level:
                        energy_to_add -= battery_state - max_battery_level  # Adjust energy added
                        battery_state = max_battery_level  # Cap battery level at 80%
                    total_energy_used_for_charging += energy_to_add  # Track energy added during charging
        
        battery_profile.append(battery_state)
    
    return availability_matrix, battery_profile, total_energy_used_for_charging, total_km_driven

# Generate profiles for each user type
results = {}
for profile_name, profile_data in profiles.items():
    availability_matrix, battery_profile, total_energy_used_for_charging, total_km_driven = generate_user_profile(
        profile_name, profile_data, days, hours_per_day
    )
    results[profile_name] = {
        "availability": availability_matrix, 
        "battery": battery_profile,
        "total_energy_used_for_charging": total_energy_used_for_charging,
        "total_km_driven": total_km_driven
    }

# Convert results to DataFrames for better visualization
for profile_name, data in results.items():
    availability_df = pd.DataFrame(data["availability"], columns=[f"Hour {i}" for i in range(hours_per_day)])
    battery_df = pd.DataFrame(data["battery"], columns=["Battery Level (kWh)"])
    
    print(f"{profile_name}'s Charging Availability:\n", availability_df)
    print(f"{profile_name}'s Battery Profile:\n", battery_df)
    print(f"Total energy used for charging {profile_name}: {data['total_energy_used_for_charging']:.2f} kWh")
    print(f"Total kilometers driven by {profile_name}: {data['total_km_driven']:.2f} km\n")


Homeoffice's Charging Availability:
    Hour 0  Hour 1  Hour 2  Hour 3  Hour 4  Hour 5  Hour 6  Hour 7  Hour 8  \
0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
1     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
2     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
3     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
4     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     0.0   
5     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   
6     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0     1.0   

   Hour 9  ...  Hour 14  Hour 15  Hour 16  Hour 17  Hour 18  Hour 19  Hour 20  \
0     0.0  ...      0.0      0.0      0.0      0.0      1.0      1.0      1.0   
1     0.0  ...      0.0      0.0      0.0      0.0      1.0      1.0      1.0   
2     0.0  ...      0.0      0.0      0.0      0.0      1.0      1.0      1.0   
3     0.0  ...      0.